https://www.kaggle.com/brendan45774/chess-piece-dectection

In [1]:
# ! pip install imageai

In [1]:
import numpy as np
import os
import shutil
from pathlib import Path

In [2]:
root_annots_path = '../data/Chess_Piece_Dectection/annotations/'
root_images_path = '../data/Chess_Piece_Dectection/images/'

annots_path = sorted([i for i in Path(root_annots_path).glob('*.xml')])
images_path = sorted([i for i in Path(root_images_path).glob('*.png')])

n_imgs = len(images_path)

classes = np.array(["Rook", "Bishop", "Pawn", "Knight", "Queen"])

In [3]:
with open(annots_path[5], 'r') as f:
    print(f.read())


<annotation>
    <folder>images</folder>
    <filename>chess13.png</filename>
    <size>
        <width>400</width>
        <height>225</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>white-queen</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>139</xmin>
            <ymin>16</ymin>
            <xmax>201</xmax>
            <ymax>154</ymax>
        </bndbox>
    </object>
</annotation>


In [4]:
os.makedirs('imageai/data/train/images', exist_ok=True)
os.makedirs('imageai/data/train/annotations', exist_ok=True)

os.makedirs('imageai/data/validation/images', exist_ok=True)
os.makedirs('imageai/data/validation/annotations', exist_ok=True)

os.makedirs('imageai/data/test/images', exist_ok=True)
os.makedirs('imageai/data/test/annotations', exist_ok=True)

In [5]:
n_imgs = 81
n_split = n_imgs // 6


for i, (annot_path, img_path) in enumerate(zip(annots_path, images_path)):    
    if i > n_imgs:
        break
    # train-val-test split
    if i < n_split:
        shutil.copy(img_path, 'imageai/data/test/images/' + img_path.parts[-1])
        shutil.copy(annot_path, 'imageai/data/test/annotations/' + annot_path.parts[-1])
    elif n_split <= i < n_split*2:
        shutil.copy(img_path, 'imageai/data/validation/images/' + img_path.parts[-1])
        shutil.copy(annot_path, 'imageai/data/validation/annotations/' + annot_path.parts[-1])
    else:
        shutil.copy(img_path, 'imageai/data/train/images/' + img_path.parts[-1])
        shutil.copy(annot_path, 'imageai/data/train/annotations/' + annot_path.parts[-1])

In [6]:
print(len(list(Path('imageai/data/train/annotations/').glob('*.xml'))))
print(len(list(Path('imageai/data/validation/annotations/').glob('*.xml'))))
print(len(list(Path('imageai/data/test/annotations/').glob('*.xml'))))

55
13
13


In [9]:
from keras.models import load_model

model = load_model("../models/pretrained-yolov3.h5")

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, None, None, 3 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, None, None, 3 0           bnorm_0[0][0]                    
____________________________________________________________________________________________

In [10]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="./imageai/data/")
trainer.setTrainConfig(object_names_array=classes,
                       batch_size=8,
                       num_experiments=10,
                       train_from_pretrained_model="../models/pretrained-yolov3.h5")
#                        train_from_pretrained_model="imageai/data/models/detection_model-ex-009--loss-0024.110.h5")

trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.81
Anchor Boxes generated.
Detection configuration saved in  ./imageai/data/json/detection_config.json
Evaluating over 0 samples taken from ./imageai/data/validation
Training over 55 samples  given at ./imageai/data/train
Some labels have no annotations! Please revise the list of labels in your configuration.
Training on: 	None
Training with Batch Size:  8


TypeError: object of type 'NoneType' has no len()